**Hi. This is part of a series of notbooks used for my Coursera IBM data Science Professional Certificate capstone project. This is week 03.**

In [85]:
import pandas as pd
import numpy as np

Get data, set up a dataframe for clean-up

In [10]:
!wget -q -O raw.html https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


In [87]:
df = pd.read_html('raw.html')
df = df[0]
x, y = df.shape

Drop 'Not assigned' Boroughs

In [88]:
droplist = []
for i in range(x):
  if df['Borough'].iloc[i] == 'Not assigned':
    droplist.append(i)
df = df.drop(droplist)
df = df.reset_index()
del df['index']

Rename unnamed neigbourhoods

In [89]:
x, y = df.shape
for i in range(x):
  if df['Neighbourhood'].iloc[i] == 'Not assigned':
    df['Neighbourhood'].iloc[i] = df['Borough'].iloc[i]

Now, we reduce the number of entries to the number of postalcodes by merging neighbourhoods.

In [90]:
droplist = []
for i in range(x):
  for j in range(x):
    if df['Postcode'].iloc[j] == df['Postcode'].iloc[i] and i > j:
      df['Neighbourhood'].iloc[i] = df['Neighbourhood'].iloc[i]+ ', ' + df['Neighbourhood'].iloc[j]
      droplist.append(j)
df = df.drop(droplist)
df = df.reset_index()
del df['index']

In [91]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


In [92]:
df.shape

(103, 3)

**Part 2**

In [9]:
!wget -q -O codes.csv https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [93]:
df_codes = pd.read_csv('codes.csv')

Find the latitude and longitude for each entry from the CSV dataframe.

In [94]:
df['Latitude'] = 'n/a'
df['Longitude'] = 'n/a'
x1, y1 = df.shape
x2, y2 = df_codes.shape
for i in range(x1):
  for j in range(x2):
    if df['Postcode'].iloc[i] == df_codes['Postal Code'].iloc[j]:
      df['Latitude'].iloc[i] = df_codes['Latitude'].iloc[j]
      df['Longitude'].iloc[i] = df_codes['Longitude'].iloc[j]


In [95]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895


**Part 3**

As per the instructions, we need to reduce the dataset just to Toronto postcodes.

In [107]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import requests
import json
from pandas.io.json import json_normalize

In [97]:
x, y = df.shape
df_toronto = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'])
for i in range(x):
  if 'oronto' in df['Borough'].iloc[i]:
    df_toronto = df_toronto.append(pd.DataFrame(df.iloc[i]).transpose())
df_toronto = df_toronto.reset_index()
del df_toronto['index']

In [98]:
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
2,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
3,M4E,East Toronto,The Beaches,43.6764,-79.293
4,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733


In [99]:
df_toronto.shape

(38, 5)

In [100]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Use Foursquare to explore the area. 

In [101]:
CLIENT_ID = 'AVHIEHE2J0LUDCC31OZKIIWWYNLJX1121GRVWSD4CBNLLXJJ' # your Foursquare ID
CLIENT_SECRET = 'VUJLZH2IPM5KACCSBU4CXE3O3I1GGVGLSVBIUWY0GQAMHTMU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


Function to pull the nearby venues from one or more given neighbourhoods. Use it to generate a dataframe of venues.

In [135]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return relevant information
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [136]:
LIMIT = 100
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King, Adelaide
Dufferin, Dovercourt Village
Union Station, Harbourfront East, Toronto Islands, Harbourfront East
Trinity, Little Portugal
Riverdale, The Danforth West
Toronto Dominion Centre, Design Exchange
Parkdale Village, Brockton, Exhibition Place, Brockton
India Bazaar, The Beaches West
Victoria Hotel, Commerce Court
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill West, Forest Hill North
The Junction South, High Park
North Toronto West
Yorkville, The Annex, North Midtown, The Annex
Roncesvalles, Parkdale
Davisville
University of Toronto, Harbord
Swansea, Runnymede
Summerhill East, Moore Park
Kensington Market, Chinatown, Grange Park, Chinatown
Summerhill West, Deer Park, Forest Hill SE, Deer Park, Rathnelly, Deer Park, Forest Hill SE, Deer Park, South Hill, Deer Park, Forest Hill SE, Deer Park, Rathnelly, Deer Park, Forest H

In [137]:
print(toronto_venues.shape)
toronto_venues.head()

(1705, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [138]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,56,56,56,56,56,56
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
Central Bay Street,81,81,81,81,81,81
Christie,17,17,17,17,17,17
Church and Wellesley,90,90,90,90,90,90
Davisville,34,34,34,34,34,34
Davisville North,7,7,7,7,7,7
"Dufferin, Dovercourt Village",15,15,15,15,15,15
"Forest Hill West, Forest Hill North",4,4,4,4,4,4


In [139]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


Analyze each neighbourhood.

In [140]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.062500
2,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012346,0.000000,...,0.000000,0.00000,0.00,0.012346,0.000000,0.000000,0.012346,0.000000,0.00,0.012346
3,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Church and Wellesley,0.011111,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.011111,0.000000,...,0.000000,0.00000,0.00,0.000000,0.011111,0.011111,0.000000,0.011111,0.00,0.011111
5,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.029412,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,"Dufferin, Dovercourt Village",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,"Forest Hill West, Forest Hill North",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,"Garden District, Ryerson",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.010000,0.000000,...,0.010000,0.00000,0.00,0.010000,0.010000,0.010000,0.010000,0.000000,0.00,0.000000


In [141]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.04
2      Farmers Market  0.04
3                Café  0.04
4         Cheese Shop  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0         Yoga Studio  0.06
1       Auto Workshop  0.06
2       Garden Center  0.06
3              Garden  0.06
4  Light Rail Station  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1                Café  0.05
2      Ice Cream Shop  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.04


----Christie----
               venue  freq
0               Café  0.18
1      Grocery Store  0.18
2               Park  0.12
3         Baby Store  0.06
4  Convenience Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.04
3           Restaurant  0.03
4              Gay Bar  0.03



In [143]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for i in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, indicators[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for i in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[i, 1:] = return_most_common_venues(toronto_grouped.iloc[i, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Steakhouse,Bakery,Seafood Restaurant,Italian Restaurant,Cheese Shop,Café
1,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Pizza Place,Comic Shop,Recording Studio,Restaurant,Burrito Place,Skate Park,Brewery,Spa
2,Central Bay Street,Coffee Shop,Ice Cream Shop,Café,Italian Restaurant,Burger Joint,Sandwich Place,Bubble Tea Shop,Gym / Fitness Center,Bakery,Chinese Restaurant
3,Christie,Grocery Store,Café,Park,Convenience Store,Diner,Candy Store,Baby Store,Athletics & Sports,Restaurant,Nightclub
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Ramen Restaurant,Gym,Gastropub,Hotel


Now we cluster the neighborhoods by k-means.

In [145]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df_toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,0,Coffee Shop,Park,Café,Pub,Bakery,Theater,Mexican Restaurant,Breakfast Spot,Yoga Studio,French Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Fast Food Restaurant,Pizza Place,Bubble Tea Shop,Middle Eastern Restaurant,Restaurant
2,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754,0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Gastropub,Breakfast Spot,Cosmetics Shop,Italian Restaurant,Bakery
3,M4E,East Toronto,The Beaches,43.6764,-79.293,0,Trail,Health Food Store,Pub,Neighborhood,Event Space,Ethiopian Restaurant,Falafel Restaurant,Electronics Store,Dim Sum Restaurant,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733,0,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Steakhouse,Bakery,Seafood Restaurant,Italian Restaurant,Cheese Shop,Café


Lastly, let's visualize the clusters.

In [147]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters